In [40]:
from search import ( # Bases para construcción de problemas
    Problem, Node, Graph, UndirectedGraph,
    SimpleProblemSolvingAgentProgram,
    GraphProblem
)

from search import ( # Algoritmos de búsqueda no informada
    tree_search, graph_search, best_first_graph_search,
    breadth_first_tree_search, breadth_first_search,
    depth_first_tree_search, depth_first_graph_search,
    depth_limited_search, iterative_deepening_search,
    uniform_cost_search,
    compare_searchers
)

from search import ( # Algoritmos de búsqueda informada (heurística)
    greedy_best_first_graph_search, astar_search
)

from math import sqrt

In [48]:
class BrokenCalc(Problem):
    """The abstract class for a formal problem.  You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""
    def __init__(self, initial = 0, goal=0, operators=["+", "*"], operands=[2, 3]):
        Problem.__init__(self, initial, goal)
        self.operands = operands
        self.operators = operators
        
        self.ops = [(operand, operator) for operand in operands for operator in operators]

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        available = list(self.ops)
        
        # Remove SQRT if the current state is a negative number
        if state < 0:
            available = list(filter(lambda x: x[1] != 'sqrt', available))
        
        return available

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        funcs = {
            '+': lambda x: state + x,
            '*': lambda x: state * x,
            '/': lambda x: state / x,
            '-': lambda x: state - x,
            'sqrt': lambda _: sqrt(state)
        }
        return funcs[action[1]](action[0])
    def h(self, node):
        "Diferencia entre meta y estado actual"
        return abs(self.goal - node.state)

#     def path_cost(self, c, state1, action, state2):
#         """Return the cost of a solution path that arrives at state2 from
#         state1 via action, assuming cost c to get up to state1. If the problem
#         is such that the path doesn't matter, this function will only look at
#         state2.  If the path does matter, it will consider c and maybe state1
#         and action. The default method costs 1 for every step in the path."""
#         return c + 1

#     def value(self, state):
#         """For optimization problems, each state has a value.  Hill-climbing
#         and related algorithms try to maximize this value."""
#         raise NotImplementedError

# print_solution
This method prints the solution to a problem. It just concatenates each action in the format `<OP> <N>` giving it as a result a string like:

$$ 0 \; OP_1 \; N_1 \; OP_2 \; N_2 \ldots = GOAL$$

Where $OP_i \in \{ +, -, *, sqrt \}$ and $N_i \in \mathbb{R}$

In [83]:
def print_solution(problem, goal):
    path = goal.solution()
    print("0", end = '')
    ops = "".join(map(lambda action: " {} {}".format(action[1], action[0]), path))
    print(ops, end = '')
    *_, res = goal.path()
    print(" = %s"  % res.state)
    print(" ==> COST: %d" % goal.path_cost)

In [84]:
p1 = BrokenCalc(goal = 20, operators=["+", "*"], operands=[2, 3])
goal1 = breadth_first_search(p1)
print("PROBLEM actions: %s" % p1.ops)
print("SOLUTION: %s" % goal1.solution())
print("PATH: %s" % goal1.path())

print_solution(p1, goal1)

PROBLEM actions: [(2, '+'), (2, '*'), (3, '+'), (3, '*')]
SOLUTION: [(2, '+'), (3, '+'), (2, '*'), (2, '*')]
PATH: [<Node 0>, <Node 2>, <Node 5>, <Node 10>, <Node 20>]
0 + 2 + 3 * 2 * 2 = 20
 ==> COST: 4


In [85]:
level_1_nums = [2, 3]
level_3_nums = [1, 6, 8]

level_1_operators = ["+", "*"]
level_3_operators = ["-", "sqrt"]

level_1_goals = [6, 7, 8, 10, 12, 15, 20, 50]
level_3_goals = [-5, 3, 5, 13, 20, 33, 82, 100]

# Level 1 solutions
The following code solves all the numbers given in the assignment description for Level 1

## BFS

In [86]:
for g in level_1_goals:
    p = BrokenCalc(goal = g, operators = level_1_operators, operands = level_1_nums)
    print("Solving for %s" % g)
    goal = breadth_first_search(p)
    print_solution(p, goal)

Solving for 6
0 + 2 * 3 = 6
 ==> COST: 2
Solving for 7
0 + 2 + 2 + 3 = 7
 ==> COST: 3
Solving for 8
0 + 2 + 2 * 2 = 8
 ==> COST: 3
Solving for 10
0 + 2 + 3 * 2 = 10
 ==> COST: 3
Solving for 12
0 + 2 + 2 * 3 = 12
 ==> COST: 3
Solving for 15
0 + 2 + 3 * 3 = 15
 ==> COST: 3
Solving for 20
0 + 2 + 3 * 2 * 2 = 20
 ==> COST: 4
Solving for 50
0 + 2 + 2 * 2 * 2 * 3 + 2 = 50
 ==> COST: 6


## Uniform Cost Search

In [88]:
for g in level_1_goals:
    p = BrokenCalc(goal = g, operators = level_1_operators, operands = level_1_nums)
    print("Solving for %s" % g)
    goal = uniform_cost_search(p)
    print_solution(p, goal)

Solving for 6
0 + 2 * 3 = 6
 ==> COST: 2
Solving for 7
0 + 2 + 2 + 3 = 7
 ==> COST: 3
Solving for 8
0 + 2 + 2 * 2 = 8
 ==> COST: 3
Solving for 10
0 + 2 + 3 * 2 = 10
 ==> COST: 3
Solving for 12
0 + 2 + 2 * 3 = 12
 ==> COST: 3
Solving for 15
0 + 2 + 3 * 3 = 15
 ==> COST: 3
Solving for 20
0 + 2 + 3 * 2 * 2 = 20
 ==> COST: 4
Solving for 50
0 + 3 * 3 + 2 * 2 + 3 * 2 = 50
 ==> COST: 6


In [ ]:
for g in level_1_goals:
    p = BrokenCalc(goal = g, operators = level_1_operators, operands = level_1_nums)
    print("Solving for %s" % g)
    goal = depth_limited_search(p)
    print_solution(p, goal)


## A* search

In [87]:
for g in level_1_goals:
    p = BrokenCalc(goal = g, operators = level_1_operators, operands = level_1_nums)
    print("Solving for %s" % g)
    goal = astar_search(p)
    print_solution(p, goal)

Solving for 6
0 + 3 * 2 = 6
 ==> COST: 2
Solving for 7
0 + 3 + 2 + 2 = 7
 ==> COST: 3
Solving for 8
0 + 3 * 2 + 2 = 8
 ==> COST: 3
Solving for 10
0 + 3 * 2 + 2 + 2 = 10
 ==> COST: 4
Solving for 12
0 + 3 * 3 + 3 = 12
 ==> COST: 3
Solving for 15
0 + 3 * 3 + 3 + 3 = 15
 ==> COST: 4
Solving for 20
0 + 3 * 3 * 2 + 2 = 20
 ==> COST: 4
Solving for 50
0 + 3 * 3 * 3 + 3 + 3 + 3 + 3 + 3 + 3 + 3 + 2 = 50
 ==> COST: 11
